In [ ]:
import torch
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torchvision.utils import save_image
from torch.autograd import Variable

import torch.nn as nn
import torch.functional as F

import os
import numpy as np
import matplotlib.pyplot as plt
import math

In [ ]:
os.makedirs('images', exist_ok=True)

In [ ]:
bs = 64
latent = 100
channels = 1
img_size = 28

In [ ]:
dataloader = DataLoader(
    datasets.MNIST('data/', download = True, transform = transforms.Compose((transforms.ToTensor(), transforms.Normalize([0.5], [0.5])))),
    bs,True
)

Extracting data/MNIST/raw/train-images-idx3-ubyte.gz to data/MNIST/raw


Extracting data/MNIST/raw/train-labels-idx1-ubyte.gz to data/MNIST/raw



Extracting data/MNIST/raw/t10k-images-idx3-ubyte.gz to data/MNIST/raw


Extracting data/MNIST/raw/t10k-labels-idx1-ubyte.gz to data/MNIST/raw
Processing...
Done!


/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:480: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:141.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [ ]:
for x,y in dataloader:
  xx = x
  print(x.shape)
  break

torch.Size([64, 1, 28, 28])


In [ ]:
cuda = True if torch.cuda.is_available() else False

In [ ]:
cuda

True

In [ ]:
# We make sure the initialized parameters are drawn from a normal distribution to help with training
def weights_init_normal(m):
  classname = m.__class__.__name__
  if classname.find("Conv") != -1:
    torch.nn.init.normal_(m.weight.data, 0.0, 0.2)
  elif classname.find("BatchNorm") != -1:
    torch.nn.init.normal_(m.weight.data, 0.0, 0.2)
    torch.nn.init.constant_(m.bias, 0.0)

In [ ]:
class Generator(nn.Module):
  def __init__(self):
    super(Generator, self).__init__()

    self.divfourthing = img_size // 4
    self.L1 = nn.Linear(latent, 128 * self.divfourthing **2)

    self.MeatOfTheModel = nn.Sequential(
        nn.BatchNorm2d(128),
        nn.Upsample(scale_factor=2),
        nn.Conv2d(128, 128, 3, 1, 1, padding_mode='reflect'),
        nn.LeakyReLU(0.2, inplace=True),
        nn.BatchNorm2d(128, 0.8),
        nn.Upsample(scale_factor=2),
        nn.Conv2d(128, 64, 3, 1, 1, padding_mode='reflect'),
        nn.LeakyReLU(0.2, inplace=True),
        nn.Conv2d(64, channels, 3, 1, 1, padding_mode='reflect'),
        nn.Tanh(),
    )
  
  def forward(self, z):
    out = self.L1(z)
    real_out = out.view(z.size(0), 128, self.divfourthing, self.divfourthing)
    return self.MeatOfTheModel(real_out)

In [ ]:
# gen = Generator()

In [ ]:
# xxx = gen(torch.randn(bs,latent))
# xxx.shape

In [ ]:
class Descriminator(nn.Module):
  def __init__(self):
    super(Descriminator, self).__init__()

    def block(inn, out, batchnorm = True):
      layers = [nn.Conv2d(inn, out, 3, 2, 1, padding_mode='reflect'), nn.LeakyReLU(0.2), nn.Dropout2d(0.25)]
      if batchnorm:
        layers.append(nn.BatchNorm2d(out))
      
      return layers
    
    self.model = nn.Sequential(
        *block(channels, 16, False),
        *block(16, 32),
        *block(32, 64),
        *block(64, 128)
    )
    
    self.thing = img_size // 2 ** 4
    self.final = nn.Sequential(nn.Linear(512, 1), nn.Sigmoid()) #Custom override.... 
  
  def forward(self,z):
    out = self.model(z)
    real_out = out.view(z.size(0), -1)
    really_out = self.final(real_out)
    return really_out

In [ ]:
# des = Descriminator()
# h = des(xxx); h.shape

In [ ]:
# real = Variable(Tensor(bs, 1).fill_(1.0), requires_grad = False)
# real.shape

In [ ]:
# h,real

In [ ]:
# los(h, real)

In [ ]:
gen = Generator()
des = Descriminator()
los = nn.BCELoss()

In [ ]:
if cuda:
  gen.cuda()
  des.cuda()
  los.cuda()

In [ ]:
gen.apply(weights_init_normal);
des.apply(weights_init_normal);

In [ ]:
sam_vec = torch.randn(bs, latent).to('cuda')

def save_imagee(jjj):
  lol = gen(sam_vec)
  lolol = lol.view(bs, channels, img_size, img_size)
  save_image(lolol, 'images/fake_image{0:0=5d}.png'.format(jjj), int(math.sqrt(bs)), normalize = True)

In [ ]:
lr = 0.00008
b1 = 0.5
b2 = 0.999

epochs = 1000
jjj = 0

loss_g_values = []
loss_d_values = []

In [ ]:
opt_g = torch.optim.Adam(gen.parameters(), lr, (b1,b2))
opt_d = torch.optim.Adam(des.parameters(), lr, (b1,b2))

In [ ]:
Tensor = torch.cuda.FloatTensor if cuda else torch.FloatTensor

In [ ]:
for __ in range(1):
  for i, (img,_) in enumerate(dataloader):
    if i % 20 == 0:
      jjj += 1
      print(jjj)
      save_imagee(jjj)
    real = Variable(Tensor(img.shape[0], 1).fill_(0.9), requires_grad = False)
    fake = Variable(Tensor(img.shape[0], 1).fill_(0.1), requires_grad = False)

    imgs = Variable(img.type(Tensor))

    opt_g.zero_grad()
    z = torch.randn(img.shape[0], latent).to('cuda')
    fake_img = gen(z)
    loss = los(des(fake_img), real)
    loss.backward()
    opt_g.step()

    opt_d.zero_grad()
    take_imgg = gen(z.detach())
    l1 = los(des(take_imgg), fake)
    l2 = los(des(imgs), real)
    losss = l1 + l2
    losss.backward()
    opt_d.step()
  print('epoch:', __)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
epoch: 0


In [ ]:
for __ in range(5):
  for i, (img,_) in enumerate(dataloader):
    if i % 50 == 0:
      jjj += 1
      print(jjj)
      save_imagee(jjj)
    real = Variable(Tensor(img.shape[0], 1).fill_(0.9), requires_grad = False)
    fake = Variable(Tensor(img.shape[0], 1).fill_(0.1), requires_grad = False)

    imgs = Variable(img.type(Tensor))

    opt_g.zero_grad()
    z = torch.randn(img.shape[0], latent).to('cuda')
    fake_img = gen(z)
    loss = los(des(fake_img), real)
    loss.backward()
    opt_g.step()

    opt_d.zero_grad()
    take_imgg = gen(z.detach())
    l1 = los(des(take_imgg), fake)
    l2 = los(des(imgs), real)
    losss = l1 + l2
    losss.backward()
    opt_d.step()
  print('epoch:', __)

48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
epoch: 0
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
epoch: 1
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
epoch: 2
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
epoch: 3
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
epoch: 4


In [ ]:
for __ in range(15):
  for i, (img,_) in enumerate(dataloader):
    if i % 360 == 0:
      jjj += 1
      print(jjj)
      save_imagee(jjj)
    real = Variable(Tensor(img.shape[0], 1).fill_(0.9), requires_grad = False)
    fake = Variable(Tensor(img.shape[0], 1).fill_(0.1), requires_grad = False)
    

    imgs = Variable(img.type(Tensor))

    opt_g.zero_grad()
    z = torch.randn(img.shape[0], latent).to('cuda')
    fake_img = gen(z)
    loss = los(des(fake_img), real)
    loss.backward()
    opt_g.step()

    opt_d.zero_grad()
    take_imgg = gen(z.detach())
    l1 = los(des(take_imgg), fake)
    l2 = los(des(imgs), real)
    losss = l1 + l2
    losss.backward()
    opt_d.step()
  print('epoch:', __)

48
49
50
epoch: 0
51
52
53
epoch: 1
54
55
56
epoch: 2
57
58
59
epoch: 3
60
61
62
epoch: 4
63
64
65
epoch: 5
66
67
68
epoch: 6
69
70
71
epoch: 7
72
73
74
epoch: 8
75
76
77
epoch: 9
78
79
80
epoch: 10
81
82
83
epoch: 11
84
85
86
epoch: 12
87
88
89
epoch: 13
90
91
92
epoch: 14
93
94
95
epoch: 15
96
97
98
epoch: 16
99
100
101
epoch: 17
102
103
104
epoch: 18
105
106
107
epoch: 19


In [ ]:
# jj = 0
# for __ in range(20):
#   run_d_loss = 0.0
#   run_g_loss = 0.0
#   for i,(img,yyy) in enumerate(dataloader):

#     run_d_loss = 0.0
#     run_g_loss = 0.0

#     if jj % 80 == 0:
#       jjj += 1
#       print(jjj)
#       save_imagee(jjj)
    
#     real = Variable(Tensor(img.shape[0], 1).fill_(0.9), requires_grad = False)
#     fake = Variable(Tensor(img.shape[0], 1).fill_(0.1), requires_grad = False)

#     imgs = Variable(img.type(Tensor))

#     z = torch.normal(0,1,size = (img.shape[0], latent)).to('cuda')

#     opt_g.zero_grad()
#     fake_img = gen(z)
#     loss = los(des(fake_img), real)
#     loss.backward()
#     opt_g.step()
#     run_d_loss =+ loss.item() * imgs.size(0)
    
#     if jj % 2 == 0:
#       opt_d.zero_grad()
#       take_imgg = gen(z.detach())
#       l1 = los(des(take_imgg), fake)
#       l2 = los(des(imgs), real)
#       losss = l1 + l2
#       losss.backward()
#       opt_d.step()

#       run_d_loss =+ losss.item() * imgs.size(0)
    
#     jj += 1
#   loss_g_values.append(run_g_loss/1000)
#   loss_d_values.append(run_d_loss / 1000)

#   print('epoch:', __)

In [ ]:
# z = torch.normal(0,1,size = (img.shape[0], latent)).to('cuda')
# h = gen(z)

In [ ]:
# h.shape, xx.shape

In [ ]:
# xxx = xx.to('cuda');

In [ ]:
# des(xxx)

In [ ]:
sample_dir = 'images'
if not os.path.exists(sample_dir):
    os.makedirs(sample_dir)

In [ ]:
#Copied
import cv2
import os
from IPython.display import FileLink

vid_fname = 'MNIST.avi'

files = [os.path.join(sample_dir, f) for f in os.listdir(sample_dir) if 'fake_image' in f]
files.sort()

out = cv2.VideoWriter(vid_fname,cv2.VideoWriter_fourcc(*'MP4V'), 16, (242,242))
[out.write(cv2.imread(fname)) for fname in files]
out.release()
FileLink('MNIST.avi')

/content/MNIST.avi